# Weekly ranking for newsletter

**Goal**: get the weekly emission rankings of 
- subnets
- validators

In [30]:
import pandas as pd
import bittensor as bt
subtensor = bt.subtensor('archive')

2024-04-24 17:55:53.412 |       INFO       | Connected to archive network and wss://archive.chain.opentensor.ai:443/.


### Subnets

In [37]:
# live_emissions
current_emissions = {}
for i in range(1, 33):
    emission = subtensor.get_emission_value_by_subnet(i)
    current_emissions[i] = float(emission)*100

current_emissions = pd.DataFrame.from_dict(current_emissions, orient='index', columns=['Emissions %'])
current_emissions = current_emissions.rename_axis('Subnet')

In [38]:
# emissions 7 days ago
# je dois aller chercher le dernier bloc et enlever 7 * 7200 = 50 400 blocks
emissions_7daysago = {}
for i in range(1, 33):
    emission = subtensor.get_emission_value_by_subnet(i, 2768078)
    emissions_7daysago[i] = float(emission)*100

emissions_7daysago = pd.DataFrame.from_dict(emissions_7daysago, orient='index', columns=['Emissions %'])
emissions_7daysago = emissions_7daysago.rename_axis('Subnet')

### Validators

Do we want a ranking of all the validators or only the root network ones ? <br>
What should we rank them with, vtrust or net return ? -> People don't care about vtrusts, only about the % returns of validators so they know where to stake

##### Method by validator
1. Get top 64 validators hotkeys from the rootnetwork metagraph
2. Look in every subnet and add how much they earn from each one/day (subnetEmissions * 0.41* dividends*7200)
3. Make the ranking

In [47]:
root_metagraph = bt.metagraph(netuid=0)

2024-04-25 15:58:44.581 |       INFO       | You are connecting to finney network with endpoint wss://entrypoint-finney.opentensor.ai:443.
2024-04-25 15:58:44.591 |     WARNING      | We strongly encourage running a local subtensor node whenever possible. This increases decentralization and resilience of the network.
2024-04-25 15:58:44.593 |     WARNING      | In a future release, local subtensor will become the default endpoint. To get ahead of this change, please run a local subtensor node and point to it.
2024-04-25 15:58:45.866 |       INFO       | Connected to finney network and wss://entrypoint-finney.opentensor.ai:443.


In [52]:
# get top 64 validators hotkeys
top_validators = root_metagraph.neurons
df = pd.DataFrame(top_validators)
top_validators_hotkeys = df.hotkey

In [61]:
metagraph = bt.metagraph(1)
neurons = pd.DataFrame(metagraph.neurons)

2024-04-25 16:29:01.062 |       INFO       | You are connecting to finney network with endpoint wss://entrypoint-finney.opentensor.ai:443.
2024-04-25 16:29:01.070 |     WARNING      | We strongly encourage running a local subtensor node whenever possible. This increases decentralization and resilience of the network.
2024-04-25 16:29:01.070 |     WARNING      | In a future release, local subtensor will become the default endpoint. To get ahead of this change, please run a local subtensor node and point to it.
2024-04-25 16:29:02.120 |       INFO       | Connected to finney network and wss://entrypoint-finney.opentensor.ai:443.


In [71]:
val = top_validators_hotkeys[3]
if val in neurons.hotkey.tolist():
    dividend = neurons.loc[neurons['hotkey'] == val, 'dividends'].iloc[0]
else:
    dividend = 0
print(dividend)

0.05482566567483024


In [92]:
daily_payout = {key: 0 for key in top_validators_hotkeys}
for i in range(1, 4):
    metagraph = bt.metagraph(i)
    emissions = float(subtensor.get_emission_value_by_subnet(i))
    neurons = pd.DataFrame(metagraph.neurons)
    for val in top_validators_hotkeys:
        if val in neurons.hotkey.tolist():
            dividends = neurons.loc[neurons['hotkey'] == val, 'dividends'].iloc[0]
            daily_payout[val] += dividends * emissions * 0.41 * 7200
        else:
            daily_payout[val] += 0
    print(daily_payout)

2024-04-25 17:21:48.214 |       INFO       | You are connecting to finney network with endpoint wss://entrypoint-finney.opentensor.ai:443.
2024-04-25 17:21:48.218 |     WARNING      | We strongly encourage running a local subtensor node whenever possible. This increases decentralization and resilience of the network.
2024-04-25 17:21:48.218 |     WARNING      | In a future release, local subtensor will become the default endpoint. To get ahead of this change, please run a local subtensor node and point to it.
2024-04-25 17:21:49.032 |       INFO       | Connected to finney network and wss://entrypoint-finney.opentensor.ai:443.
{'5CaCUPsSSdKWcMJbmdmJdnWVa15fJQuz5HsSGgVdZffpHAUa': 0, '5E7v9fjMkA1rSchQiaz9gX1vg62fFz8HrAVipkzpmo1ZRsgG': 0.6593076265939116, '5F4tQyWrhfGVcNhoqeiNsR6KjD4wMZ2kfhLj4oHYuyHbZAc3': 38.40590854914534, '5FFApaS75bv5pJHfAp2FVLBj9ZaXuFDjEypsaBNc1wCfe52v': 8.895695758817853, '5HK5tp6t2S59DywmHRWPBVJeJ86T61KjurYqeooqj8sREpeN': 12.82923411748153, '5CXRfP2ekFhe62r7q3vppRa

In [122]:
validators_df = pd.DataFrame(daily_payout, index=[0]).T
validators_df = validators_df.rename(columns={0: 'daily reward'})
validators_df = validators_df.rename_axis('hotkey')

In [123]:
stake_df = df[['hotkey', 'total_stake']].set_index('hotkey')
# I need a way to extract the stake from the total_stake column 
stake_df['total_stake'] = [float(element) for element in stake_df.total_stake.tolist()]

In [124]:
validators_df = validators_df.join(stake_df)

In [126]:
validators_df['APR'] = ((validators_df['daily reward'] * 365) / validators_df['total_stake'])*100